1. CONNECT WITH OPENAI PLATFORM AND TEST INITIAL FILE FOLDER DIRECTORY

In [1]:
import sys
sys.path.append('C:/Users/mlshe/SWE559/volunteer-finder')

In [2]:
from config import Config
from openai import OpenAI

Config.validate()
client = Config.get_openai_client()

print(f"Using model: {Config.MODEL_NAME}")
print(f"Max Tokens: {Config.MAX_COMPLETION_TOKENS}")
print(f"Hurray Setup complete! Ready to build")

Using model: gpt-4o-mini
Max Tokens: 2100
Hurray Setup complete! Ready to build


In [3]:
def test_gpt_4o_mini_model():
    try:
        check = client.chat.completions.create(
            model = Config.MODEL_NAME,
            messages=[
                {"role": "user", "content": "Say 'gpt-4o-mini is ready for resume analysis!' if you receive this."}
            ],
            max_completion_tokens = Config.MAX_COMPLETION_TOKENS
        )
        print("=" * 65)
        print("GPT-4o-mini Connection Test:")
        print("=" * 65)
        print(check.choices[0].message.content)
        print("Usage Stats:")
        print(f"Prompt Tokens: {check.usage.prompt_tokens}")
        print(f"Completion Tokens: {check.usage.completion_tokens}")
        print(f"Reasoning: {check.usage.completion_tokens_details.reasoning_tokens}")
        print(f"Total Tokens: {check.usage.total_tokens}")
        print("=" * 60)
        print("Ready for Lesson 2")
        return True
    
    except Exception as e:
        print(f"Connection failed: {e}")
        import traceback
        traceback.print_exc
        return False

response = test_gpt_4o_mini_model()

GPT-4o-mini Connection Test:
gpt-4o-mini is ready for resume analysis!
Usage Stats:
Prompt Tokens: 26
Completion Tokens: 12
Reasoning: 0
Total Tokens: 38
Ready for Lesson 2


2. CREATE A CLASS TO READ A FILE AND THEN EXTRACT INFORMATION INTO A VECTOR --> Sent to a separate .py file

In [15]:
code = '''
"""
File reader
"""

import os
from pathlib import Path
from typing import Optional
import PyPDF2
from docx import Document

class ResumeReader:
    """Interface for reading resume formats"""

    supported_formats = ['.pdf', '.docx', '.txt']

    def read_file(file_path):
        """
        Read a file and return its text content

        Args:
            file_path: Path to the resume file

        returns:
            Extracted text content or None if failed
        """

        file_path = Path(file_path)
        if not file_path.exists():
            print(f"File not found: {file_path}")
            return None
        
        extension = file_path.suffix.lower()
        if extension not in ResumeReader.supported_formats:
            print(f" Unsupported format: {extension}")
            print(f"Supported formats: {', '.join(ResumeReader.supported_formats)}")
            return None
        
        try:
            if extension == '.txt':
                return ResumeReader.read_txt(file_path)
            elif extension == '.docx':
                return ResumeReader.read_docx(file_path)
            elif extension == '.pdf':
                return ResumeReader.read_pdf(file_path)
        except Exception as e:
            print(f'Error reading {file_path.name}: {e}')
            return None 
        
    def read_txt(file_path):
        with open(file_path, 'r', encoding='utf-8') as txt_file:
            text = txt_file.read()
        print(f"Read TXT: {file_path.name} with ({len(text)} of words to process)")
        return text
    
    def read_docx(file_path):
        doc = Document(file_path)
        text_doc = []
        for paragraph in doc.paragraphs:
            if paragraph.text.strip():
                text_doc.append(paragraph.text)

        for table in doc.tables:
            for row in table.rows:
                for cell in table.cells:
                    if cell.text.strip():
                        text_doc.append(cell.text)
        
        text = '//n'.join(text_doc)
        print(f"Read docx: {file_path.name} with ({len(text)}) of words to process")

        def read_pdf(file_path):
            text_parts = []
            with open(file_path, 'rb') as file_pdf:
                pdf_reader = PyPDF2.PdfReader(file_pdf)
                num_pages = len(pdf_reader.pages)
                for pages in range(num_pages):
                    page = pdf_reader.pages[pages]
                    text = page.extract_text()
                    if text.strip():
                        text_parts.append(text)
            full_text = '//n'.join(text_parts)
            print(f"Read PDF: {file_path.name} with ({len(full_text)} of words ready to process from {num_pages} pages)")
            return full_text
        
        def get_file_information(file_path):
            file_path = Path(file_path)

            if not file_path.exists():
                return {"Error": "File Not Found"}
            
            return {
                "filename": file_path.name,
                "extension": file_path.suffix.lower(),
                "supported": file_path.suffix.lower() in ResumeReader.supported_formats
            }
'''
with open('C:/Users/mlshe/SWE559/volunteer-finder/file_readers.py', 'w') as file:
    file.write(code)

print(f"Created file_readers.py")

Created file_readers.py


In [4]:
import os
from pathlib import Path

resume_folder = 'C:/Users/mlshe/SWE559/volunteer-finder/sample_resumes'

In [5]:
print("="*50)
print(f"Scanning folder: {resume_folder}")
print("="*50)

all_files = list(Path(resume_folder).glob('*'))
print(f"Total files found: {len(all_files)}")

Scanning folder: C:/Users/mlshe/SWE559/volunteer-finder/sample_resumes
Total files found: 50


In [6]:
from collections import Counter
extensions = Counter([f.suffix.lower() for f in all_files if f.is_file()])

print("File Types")
for exten, count in extensions.items():
    supported_file = "Good" if exten in ['.pdf', '.docx', '.txt'] else 'Not_Good'
    print(f"{supported_file} {exten or 'no extensions'}: {count} files")

print("="*50)
print("First 10 files in the Directory")
for info, file in enumerate(all_files[:10], 1):
    print(f"{info}.{file.name}")
print("="*50)

File Types
Good .txt: 17 files
Good .docx: 15 files
Good .pdf: 18 files
First 10 files in the Directory
1.Aimee_Montoya_Resume.txt
2.Allison_Hill_Resume.txt
3.Andrew_Graham_Resume.docx
4.Angela_Dennis_Resume.txt
5.Angela_Hernandez_Resume.pdf
6.Ashley_James_Resume.pdf
7.Brittany_Anderson_Resume.pdf
8.Cameron_Dean_Resume.docx
9.Cameron_Fisher_Resume.txt
10.Cindy_Anderson_Resume.pdf


In [7]:
import sys
sys.path.append("..")

from file_readers import ResumeReader
from pathlib import Path
import time

print('ResumeReader imported successfully')
print(f'Supported formats: {ResumeReader.SUPPORTED_FORMATS}')

ResumeReader imported successfully
Supported formats: ['.pdf', '.docx', '.txt']


In [8]:
resume_folder

'C:/Users/mlshe/SWE559/volunteer-finder/sample_resumes'

In [9]:
def process_resumes_data(folder_path):
    """
    Proess all resumes in the folder
    """
    folder = Path(folder_path)

    results = {'successful': [],
              'failed': [],
              'resume_data': {},
              'stats':{
                  'total_files': 0,
                  'total_characters': 0,
                  'total_words': 0,
                  'avg_length': 0
              }

    }

    print("="*50)
    print("Processing Resumes....")
    print("="*50)

    start_time = time.time()

    all_files = list(folder.glob('*'))

    for file_path in all_files:
        if file_path.is_dir():
            continue

        if file_path.suffix.lower() not in ResumeReader.SUPPORTED_FORMATS:
            continue

        results['stats']['total_files'] += 1

        print(f"Reading: {file_path.name}")
        text = ResumeReader.read_file(str(file_path))

        if text and len(text.strip()) > 0:
            results['successful'].append(file_path.name)
            results['resume_data'][file_path.name] = text

            char_count = len(text)
            word_count = len(text.split())

            results['stats']['total_characters'] += char_count
            results['stats']['total_words'] += word_count

            print(f"{char_count:,} characters, {word_count:,} words/n")
        
        else:
            results['failed'].append(file_path.name)
            print(f"Failed/n")
    
    if results['successful']:
        results['stats']['avg_length'] = results['stats']['total_characters'] // len(results['successful'])

    elapsed_time = time.time() - start_time

    print(f"/n" + "="*50)
    print("Processing Summary")
    print("="*50)
    print(f"Successful: {len(results['successful'])} / {results['stats']['total_files']}")
    print(f"Failed: {len(results['failed'])}")
    print(f"Total characters: {results['stats']['total_characters']:,}")
    print(f"Total words: {results['stats']['total_words']:,}")
    print(f"Average length: {results['stats']['avg_length']:,} characters")
    print(f"Time: {elapsed_time:.2f} seconds")
    print("=" * 50)
    
    if results['failed']:
        print("/nFailed files:")
        for filename in results['failed']:
            print(f"   - {filename}")
    
    return results

resume_data1 = process_resumes_data(resume_folder)

Processing Resumes....
Reading: Aimee_Montoya_Resume.txt
645 characters, 77 words/n
Reading: Allison_Hill_Resume.txt
679 characters, 80 words/n
Reading: Andrew_Graham_Resume.docx
652 characters, 78 words/n
Reading: Angela_Dennis_Resume.txt
648 characters, 77 words/n
Reading: Angela_Hernandez_Resume.pdf
629 characters, 77 words/n
Reading: Ashley_James_Resume.pdf
640 characters, 76 words/n
Reading: Brittany_Anderson_Resume.pdf
666 characters, 79 words/n
Reading: Cameron_Dean_Resume.docx
645 characters, 76 words/n
Reading: Cameron_Fisher_Resume.txt
617 characters, 73 words/n
Reading: Cindy_Anderson_Resume.pdf
622 characters, 76 words/n
Reading: Cody_Larson_Resume.pdf
644 characters, 78 words/n
Reading: Courtney_Keller_Resume.txt
648 characters, 78 words/n
Reading: Deborah_Figueroa_Resume.docx
665 characters, 80 words/n
Reading: Destiny_Strickland_Resume.txt
620 characters, 76 words/n
Reading: Dr._Steven_Martin_Resume.pdf
626 characters, 77 words/n
Reading: Edward_Novak_Resume.txt
636 char

In [22]:
text_processor = '''
class TextProcessor:
    def clean_whitespace(text):
        text = text.replace('//t', ' ') 
        words = text.split()
        text = ' '.join(words)

        text = text.strip()

        return text
    
    def clean_newlines(text):
        while '//n//n//n' in text:
            text = text.replace('//n//n//n', '//n//n')

        return text
    
    def remove_special_characters(text):
        allowed = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789 //n.,!?-:;()@#$%&+='
        cleaned = ''
        for char in text:
            if char is allowed:
                cleaned += char
        return cleaned
    
    def to_lowercase(text):
        return text.lower()
    
    def find_email(text):
        words = text.split()
        for word in words:
            if'@' in word and '.' in word:
                email = word.strip('.,;:!?()')
                return email
        return None
    
    def find_phone(text):
        words = text.split()
        for word in words:
            digit_count = sum(1 for char in word if char.isdigit())
            if digit_count == 10:
                return word
        return None
    
    def has_section(text, section_name):
        text_lower = text.lower()
        section_lower = section_name.lower()
        return section_lower in text_lower
    
    def preprocess(text):
        """
        Pre-processing steps:
        1) Clean whitespace
        2) Clean newlines
        3) Remove special characters
        4) Clean whitespace again (after removing special characters)
        """
        text = TextProcessor.clean_whitespace(text)
        text = TextProcessor.clean_newlines(text)
        text = TextProcessor.remove_special_characters(text)
        text = TextProcessor.clean_whitespace(text)

        return text
'''
with open('C:/Users/mlshe/SWE559/volunteer-finder/text_processor.py', 'w') as output_file:
    output_file.write(text_processor)

print("Check file directory. Text_Processor should be available")


Check file directory. Text_Processor should be available


Created Text_Processor.py file. Completed testing below for general checks

In [10]:
# test_text_new = """
# John Doe
# Software Engineer
# Email: john.doe@example.com
# Phone: 5551234567

# Experience:
# - Worked at Tech Corp
# - Built cool things
# """

import sys
sys.path.append('..')

from text_processor import TextProcessor

# # Test 1 Clean Whitespace
# print("Clean Whitespace")
# test_text = "Python    JavaScript     SQL"
# cleaned = TextProcessor.clean_whitespace(test_text)
# print(f"   Before: '{test_text}'")
# print(f"   After:  '{cleaned}'/n")

# # Test 2 Find Email
# print("Find Email")
# email = TextProcessor.find_email(test_text_new)
# print(f"Found: {email if email else 'Not Found'}/n")

# # Test 3 Find Phone Number
# print("Find Phone Number")
# phone = TextProcessor.find_phone(test_text_new)
# if phone:
#     print(f"Found Number: {phone}")
# else:
#     print("No Number")

# Test on an actual resume
print("Test on a Resume")
sample_name = list(resume_data1['resume_data'].keys())[4]
sample_text = resume_data1['resume_data'][sample_name]

email = TextProcessor.find_email(sample_text)
phone = TextProcessor.find_phone(sample_text)

print(f"Resume: {sample_name}")
print(f"Email: {email if email else 'Email not found'}")
print(f"Phone: {phone if phone else 'Phone Number not found'}")


Test on a Resume
Resume: Angela_Hernandez_Resume.pdf
Email: tbenson@umich.edu
Phone: (734) 937-5920


In [11]:
def process_all_resumes(resume_data):
    """
    Process all resumes with error handling.
    
    Args:
        resume_data: Dictionary containing resume data
        
    Returns:
        dict: Processed resume data with original, cleaned, and metadata
    """
    processed = {
        'original': {},
        'cleaned': {},
        'metadata': {}
    }
    
    failed_resumes = []  # Track failures

    print("=" * 60)
    print("Processing all Resumes")
    print("=" * 60)

    success_count = 0
    email_count = 0
    phone_count = 0
    total_resumes = len(resume_data['resume_data'])

    for idx, (filename, text) in enumerate(resume_data['resume_data'].items(), 1):
        try:
            # Validate input
            if not text or not isinstance(text, str):
                raise ValueError(f"Invalid text data for {filename}")
            
            if len(text.strip()) == 0:
                raise ValueError(f"Empty resume text for {filename}")
            
            # Clean text
            cleaned = TextProcessor.preprocess(text, remove_special=False)

            # Extract metadata
            email = TextProcessor.find_email(text)
            phone = TextProcessor.find_phone(text)
            stats = TextProcessor.get_stats(cleaned)

            # Check sections
            has_experience = TextProcessor.has_section(text, 'experience')
            has_skills = TextProcessor.has_section(text, 'skills')
            has_education = TextProcessor.has_section(text, 'education')

            # Store results
            processed['original'][filename] = text
            processed['cleaned'][filename] = cleaned
            processed['metadata'][filename] = {
                'email': email,
                'phone': phone,
                'has_experience': has_experience,
                'has_skills': has_skills,
                'has_education': has_education,
                'stats': stats
            }
            
            # Count successes
            success_count += 1
            if email:
                email_count += 1
            if phone:
                phone_count += 1
            
            # Print progress every 10 resumes
            if success_count % 10 == 0:
                print(f"   Processed {success_count}/{total_resumes} resumes...")
        
        except Exception as e:
            # Log the error but continue processing
            error_msg = f"{filename}: {str(e)}"
            failed_resumes.append(error_msg)
            print(f"   ⚠️  Error processing {filename}: {str(e)}")
            continue

    print("/n" + "=" * 60)
    print("PROCESSING COMPLETE")
    print("=" * 60)
    print(f"Total processed: {success_count}/{total_resumes}")
    print(f"Emails found: {email_count}/{total_resumes} ({email_count/total_resumes*100:.0f}%)")
    print(f"Phones found: {phone_count}/{total_resumes} ({phone_count/total_resumes*100:.0f}%)")
    
    if failed_resumes:
        print("/n" + "=" * 60)
        print(f"FAILED RESUMES: {len(failed_resumes)}")
        print("=" * 60)
        for failure in failed_resumes:
            print(f"   ✗ {failure}")
    
    print("=" * 60)
    
    return processed


# Run the processing
final_processed = process_all_resumes(resume_data1)

# Display results
if final_processed['metadata']:
    first_filename = list(final_processed['metadata'].keys())[0]
    print(f"/nExample of structure for: {first_filename}")
    print(f"Metadata Keys: {list(final_processed['metadata'][first_filename].keys())}")
    print(f"Meta data check 'education': {final_processed['metadata'][first_filename]['has_education']}")
    print(f"/nFirst 200 chars of cleaned resume:")
    print(final_processed['cleaned'][first_filename][:200])
else:
    print("⚠️  No resumes were successfully processed!")
         

Processing all Resumes
   Processed 10/50 resumes...
   Processed 20/50 resumes...
   Processed 30/50 resumes...
   Processed 40/50 resumes...
   Processed 50/50 resumes...
/n============================================================
PROCESSING COMPLETE
Total processed: 50/50
Emails found: 50/50 (100%)
Phones found: 50/50 (100%)
/nExample of structure for: Aimee_Montoya_Resume.txt
Metadata Keys: ['email', 'phone', 'has_experience', 'has_skills', 'has_education', 'stats']
Meta data check 'education': True
/nFirst 200 chars of cleaned resume:
Name: Aimee Montoya Degree: Bachelor of Science in Psychology Email: sarah12@umich.edu Phone: (734) 492-7930 LinkedIn: https://www.linkedin.com/in/darrell18 Summary: Motivated Psychology student passi


In [12]:
from config import Config

Config.validate()
client = Config.get_openai_client()

print("="*50)
print("OpenAI Configuration")
print("="*50)
print(f"Model: {Config.MODEL_NAME}")
print(f"Max Completion Tokens: {Config.MAX_COMPLETION_TOKENS}")
print(f"Temperature: {Config.TEMPERATURE}")
print(f"Chunk Size: {Config.CHUNK_SIZE}")
print(f"Overlap: {Config.CHUNK_OVERLAP}")
print(f"="*50)

OpenAI Configuration
Model: gpt-4o-mini
Max Completion Tokens: 2100
Temperature: 1.0
Chunk Size: 2000
Overlap: 200


In [13]:
import tiktoken

# Get tokenizer (use gpt-4 encoding as fallback for gpt-5-mini)
encoding = tiktoken.encoding_for_model("gpt-4")

test_text = "I have 0 years of experience with Python, JavaScript, and team leadership. What should I do?"

tokens = encoding.encode(test_text)
print("=" * 60)
print("🔢 Token Analysis")
print("=" * 60)
print(f"Text: '{test_text}'")
print(f"Characters: {len(test_text)}")
print(f"Estimated tokens: {len(tokens)}")
print(f"Ratio: ~{len(test_text)/len(tokens):.1f} chars per token")

🔢 Token Analysis
Text: 'I have 0 years of experience with Python, JavaScript, and team leadership. What should I do?'
Characters: 92
Estimated tokens: 21
Ratio: ~4.4 chars per token


In [14]:
# Cell 3: First GPT-4o-mini API Call with Full Token Breakdown
print("=" * 60)
print("🤖 Testing GPT-4o-mini with Token Analysis")
print("=" * 60)

test_prompt = """
Analyze this resume snippet and provide a score from 1-10.

Resume: "Community Volunteer Coordinator with 2 years experience. 
Bachelor's in Psychology. Skilled in team leadership and SPSS."

Format:
Score: [number]
Reason: [one sentence]
"""

print("📤 Sending to GPT-4o-mini.../n")

try:
    response = client.chat.completions.create(
        model=Config.MODEL_NAME,
        messages=[
            {"role": "system", "content": "You are an expert resume analyst."},
            {"role": "user", "content": test_prompt}
        ],
        max_completion_tokens=200,
        temperature=Config.TEMPERATURE
    )
    
    print("✅ Response received!/n")
    print("📥 AI Response:")
    print("-" * 60)
    print(response.choices[0].message.content)
    print("-" * 60)
    
    # Detailed token breakdown for GPT-5-Mini
    print(f"/n📊 Token Usage Breakdown:")
    print(f"   Input (prompt) tokens: {response.usage.prompt_tokens}")
    print(f"   Output (response) tokens: {response.usage.completion_tokens}")
    
    # Check if reasoning tokens exist
    if hasattr(response.usage.completion_tokens_details, 'reasoning_tokens'):
        reasoning = response.usage.completion_tokens_details.reasoning_tokens
        actual_output = response.usage.completion_tokens - reasoning
        print(f"   └─ Reasoning tokens: {reasoning} (internal thinking)")
        print(f"   └─ Actual output tokens: {actual_output} (visible response)")
    
    print(f"   TOTAL tokens: {response.usage.total_tokens}")
    
    print(f"/n💡 Token Insight:")
    print(f"   GPT-4o-mini used {response.usage.total_tokens} total tokens")
    print(f"   This includes 'thinking' time before responding")
    
except Exception as e:
    print(f"❌ Error: {e}")

🤖 Testing GPT-4o-mini with Token Analysis
📤 Sending to GPT-4o-mini.../n
✅ Response received!/n
📥 AI Response:
------------------------------------------------------------
Score: 6  
Reason: The resume snippet provides basic qualifications and skills but lacks specificity about achievements, responsibilities, or the impact of the volunteer work, which could enhance its effectiveness.
------------------------------------------------------------
/n📊 Token Usage Breakdown:
   Input (prompt) tokens: 72
   Output (response) tokens: 37
   └─ Reasoning tokens: 0 (internal thinking)
   └─ Actual output tokens: 37 (visible response)
   TOTAL tokens: 109
/n💡 Token Insight:
   GPT-4o-mini used 109 total tokens
   This includes 'thinking' time before responding


In [15]:
from file_readers import ResumeReader
from pathlib import Path

sample_oppportunities = 'C:/Users/mlshe/SWE559/volunteer-finder/sample_opportunities'

print('='*50)
print('Load Job Descriptions')
print('='*50)

opportunity_folder = Path(sample_oppportunities)
if not opportunity_folder.exists():
    print(f"Folder not found: {sample_oppportunities}")
    print(f"Please update pathway")
else:
    print(f"Found folder: {sample_oppportunities}/n")
    
    job_data = {
        'jobs': {},
        'stats': {
            'total': 0,
            'successful': 0,
            'failed': 0
        }
    }
    all_files = sorted(opportunity_folder.glob('Volunteer_Role_*'))

    for file_path in all_files:
        if file_path.is_dir():
            continue

        if file_path.suffix.lower() not in ResumeReader.SUPPORTED_FORMATS:
            continue

        job_data['stats']['total'] += 1
        print(f"Reading file: {file_path.name}")

        text = ResumeReader.read_file(str(file_path))
        if text and len(text.strip()) > 0:
            job_data['jobs'][file_path.name] = text
            job_data['stats']['successful'] += 1
        else:
            job_data['stats']['failed'] += 1

    print("/n" + "=" * 60)
    print("JOB POSTINGS LOADED")
    print("=" * 60)
    print(f"Successfully loaded: {job_data['stats']['successful']}")
    print(f"Failed: {job_data['stats']['failed']}")
    print("=" * 60)
    
    if job_data['stats']['successful'] > 0:
        print(f"/nLoaded job postings:")
        for i, filename in enumerate(sorted(job_data['jobs'].keys()), 1):
            # Extract position name from the job description
            job_text = job_data['jobs'][filename]
            first_line = job_text.split('/n')[0]
            print(f"   {i}. {filename} - {first_line[:60]}...")



Load Job Descriptions
Found folder: C:/Users/mlshe/SWE559/volunteer-finder/sample_opportunities/n
Reading file: Volunteer_Role_1.pdf
Reading file: Volunteer_Role_10.pdf
Reading file: Volunteer_Role_100.pdf
Reading file: Volunteer_Role_11.pdf
Reading file: Volunteer_Role_12.pdf
Reading file: Volunteer_Role_13.pdf
Reading file: Volunteer_Role_14.pdf
Reading file: Volunteer_Role_15.pdf
Reading file: Volunteer_Role_16.pdf
Reading file: Volunteer_Role_17.pdf
Reading file: Volunteer_Role_18.pdf
Reading file: Volunteer_Role_19.pdf
Reading file: Volunteer_Role_2.pdf
Reading file: Volunteer_Role_20.pdf
Reading file: Volunteer_Role_21.pdf
Reading file: Volunteer_Role_22.pdf
Reading file: Volunteer_Role_23.pdf
Reading file: Volunteer_Role_24.pdf
Reading file: Volunteer_Role_25.pdf
Reading file: Volunteer_Role_26.pdf
Reading file: Volunteer_Role_27.pdf
Reading file: Volunteer_Role_28.pdf
Reading file: Volunteer_Role_29.pdf
Reading file: Volunteer_Role_3.pdf
Reading file: Volunteer_Role_30.pdf
Read

In [16]:
def extract_job_info(job_text):
    info = {
        'position': '',
        'department': '',
        'volunteers_needed': '',
        'requirements': [],
        'preferred': [],
        'responsibilities': []
    }

    lines = job_text.split('/n')

    for line in lines:
        line = line.strip()

        if 'POSITION:' in line:
            info['position'] = line.replace('POSITION:', '').strip()

        elif 'DEPARTMENT:' in line:
            info['department'] = line.replace('DEPARTMENT:', '').strip()

        elif 'VOLUNTEERS_NEEDED:' in line:
            info['volunteers_needed'] = line.replace('VOLUNTEERS NEEDED:', '').strip()

        elif line.startswith('✓') or line.startswith('•') or (len(line) > 0 and line[0] in ['√', '☑', '✔']):
            cleaned = line[1:].strip()
            if cleaned and len(info['requirements']) < 10:  # Limit to prevent over-collection
                info['requirements'].append(cleaned)
    return info

if job_data['stats']['successful'] > 0:
    print('='*50)
    print('Extracting Job Details')
    print('='*50)

    for filename in job_data['jobs'].keys():
        job_info = extract_job_info(job_data['jobs'][filename])
        job_data['jobs'][filename] = {
            'full_text': job_data['jobs'][filename],
            'info': job_info
        }
        print(f"/n{filename}")
        print(f"   Position: {job_info['position']}")
        print(f"   Department: {job_info['department']}")
        print(f"   Volunteers Needed: {job_info['volunteers_needed']}")
        print(f"   Requirements found: {len(job_info['requirements'])}")
    
    print("/n" + "=" * 60)

Extracting Job Details
/nVolunteer_Role_1.pdf
   Position: Community Outreach Support Volunteer
DEPARTMENT: Mathematics
Volunteers Needed: 2
ORGANIZATIONAL LEADERS:
- Dr. Andrew Parker
- Prof. John Carter
- Dr. Olivia Scott
ABOUT THE ROLE:
In this role, you will support departmental initiatives and foster community engagement. Volunteers will contribute to improving student experience and supporting academic success.
REQUIREMENTS:
✓ Enrolled in or completed a relevant academic program
✓ Strong communication and collaboration skills
✓ Ability to work with diverse student populations
✓ Reliable and organized with strong time management
PREFERRED QUALIFICATIONS:
✓ Leadership experience or past volunteer work
✓ Familiarity with campus resources and student groups
✓ Experience with event support or mentoring
RESPONSIBILITIES:
 Assist students, staff, or faculty depending on assignment
 Support program operations and outreach activities
 Track engagement and provide feedback to supervisor

In [17]:
def match_candidates_to_volunteer_role(resumes, job_data, job_filename, max_candidates):
    """
    Match candidates to volunteer role with 100-point scoring
    """
    
    job = job_data['jobs'][job_filename]
    job_text = job['full_text']
    job_info = job['info']
    
    results = []
    total_tokens = 0
    
    print("=" * 60)
    print(f"MATCHING CANDIDATES TO VOLUNTEER ROLE")
    print("=" * 60)
    print(f"Position: {job_info['position']}")
    print(f"Department: {job_info['department']}")
    print(f"Volunteers Needed: {job_info['volunteers_needed']}")
    print(f"Candidates to screen: {min(max_candidates, len(resumes['cleaned']))}")
    print("=" * 60)
    
    candidates = list(resumes['cleaned'].keys())[:max_candidates]
    
    for i, filename in enumerate(candidates, 1):
        print(f"/n[{i}/{len(candidates)}] {filename[:45]}...")
        
        resume_text = resumes['cleaned'][filename]
        
        # Scoring prompt with 100-point scale
        prompt = f"""
You are screening volunteer candidates. Score this candidate objectively using a 100-point scale.

VOLUNTEER POSITION:
{job_text}

CANDIDATE RESUME:
{resume_text}

Provide scores in this EXACT format (use numbers 1-100):

OVERALL_SCORE: [number 1-100]
EDUCATION_MATCH: [number 1-100]
EXPERIENCE_MATCH: [number 1-100]
SKILLS_MATCH: [number 1-100]
COMMUNICATION_FIT: [number 1-100]
KEY_STRENGTH: [one specific strength - max 12 words]
MAIN_CONCERN: [one specific concern - max 12 words]
RECOMMENDATION: [STRONG_FIT/POTENTIAL_FIT/WEAK_FIT]
BRIEF_REASONING: [2 sentences max - why this score?]
"""
        
        try:
            response = client.chat.completions.create(
                model=Config.MODEL_NAME,
                messages=[
                    {"role": "system", "content": "You are a volunteer coordinator. Use the full 1-100 scale. 90-100=excellent, 70-89=good, 50-69=acceptable, below 50=poor fit."},
                    {"role": "user", "content": prompt}
                ],
                max_completion_tokens=1000,
                temperature=0.5
            )
            
            analysis = response.choices[0].message.content
            
            result = {
                'filename': filename,
                'analysis': analysis,
                'tokens': response.usage.total_tokens,
                'metadata': resumes['metadata'][filename]
            }
            
            results.append(result)
            total_tokens += response.usage.total_tokens
            
            # Extract and show score immediately
            import re
            score_match = re.search(r'OVERALL_SCORE:/s*(/d+)', analysis)
            if score_match:
                score = score_match.group(1)
                print(f"Score: {score}/100")
            else:
                print(f"Scored")
            
            # Small delay
            import time
            time.sleep(0.3)
            
        except Exception as e:
            print(f"Error: {e}")
    
    print("/n" + "=" * 60)
    print("SCREENING COMPLETE")
    print("=" * 60)
    print(f"Screened: {len(results)}/{len(candidates)} candidates")
    print(f"Total tokens: {total_tokens:,}")
    print("=" * 60)
    
    return {
        'job_filename': job_filename,
        'job_info': job_info,
        'job_text': job_text,
        'results': results,
        'summary': {
            'total_screened': len(results),
            'total_tokens': total_tokens
        }
    }

# Run the matching
if job_data['stats']['successful'] > 0:
    selected_job = sorted(job_data['jobs'].keys())[0]
    
    print(f"/nSelected: {selected_job}")
    print(f"Position: {job_data['jobs'][selected_job]['info']['position']}/n")
    
    # Match all candidates
    matching_results = match_candidates_to_volunteer_role(
        final_processed,
        job_data,
        selected_job,
        max_candidates=50
    )

/nSelected: Volunteer_Role_1.pdf
Position: Community Outreach Support Volunteer
DEPARTMENT: Mathematics
Volunteers Needed: 2
ORGANIZATIONAL LEADERS:
- Dr. Andrew Parker
- Prof. John Carter
- Dr. Olivia Scott
ABOUT THE ROLE:
In this role, you will support departmental initiatives and foster community engagement. Volunteers will contribute to improving student experience and supporting academic success.
REQUIREMENTS:
✓ Enrolled in or completed a relevant academic program
✓ Strong communication and collaboration skills
✓ Ability to work with diverse student populations
✓ Reliable and organized with strong time management
PREFERRED QUALIFICATIONS:
✓ Leadership experience or past volunteer work
✓ Familiarity with campus resources and student groups
✓ Experience with event support or mentoring
RESPONSIBILITIES:
 Assist students, staff, or faculty depending on assignment
 Support program operations and outreach activities
 Track engagement and provide feedback to supervisors
 Participate 

In [18]:
import re

def parse_candidate_analysis(analysis_text):
    """
    Extract all information from AI analysis including reasoning
    """
    
    parsed = {
        'scores': {
            'overall': 0,
            'education': 0,
            'experience': 0,
            'skills': 0,
            'communication': 0
        },
        'recommendation': 'UNKNOWN',
        'key_strength': 'Not provided',
        'main_concern': 'Not provided',
        'reasoning': 'Not provided',
        'full_analysis': analysis_text,
        'grade': '',
        'pass_threshold': False
    }
    
    # Extract scores
    score_patterns = {
        'overall': r'OVERALL_SCORE:/s*(/d+)',
        'education': r'EDUCATION_MATCH:/s*(/d+)',
        'experience': r'EXPERIENCE_MATCH:/s*(/d+)',
        'skills': r'SKILLS_MATCH:/s*(/d+)',
        'communication': r'COMMUNICATION_FIT:/s*(/d+)'
    }
    
    for key, pattern in score_patterns.items():
        match = re.search(pattern, analysis_text, re.IGNORECASE)
        if match:
            parsed['scores'][key] = int(match.group(1))
    
    # Extract recommendation
    rec_match = re.search(r'RECOMMENDATION:/s*([A-Z_]+)', analysis_text, re.IGNORECASE)
    if rec_match:
        parsed['recommendation'] = rec_match.group(1).strip()
    
    # Extract key strength
    strength_match = re.search(r'KEY_STRENGTH:/s*(.+?)(?=/n[A-Z_]+:|$)', analysis_text, re.IGNORECASE | re.DOTALL)
    if strength_match:
        parsed['key_strength'] = strength_match.group(1).strip()
    
    # Extract main concern
    concern_match = re.search(r'MAIN_CONCERN:/s*(.+?)(?=/n[A-Z_]+:|$)', analysis_text, re.IGNORECASE | re.DOTALL)
    if concern_match:
        parsed['main_concern'] = concern_match.group(1).strip()
    
    # Extract reasoning
    reasoning_match = re.search(r'BRIEF_REASONING:/s*(.+?)(?=/n[A-Z_]+:|$)', analysis_text, re.IGNORECASE | re.DOTALL)
    if reasoning_match:
        parsed['reasoning'] = reasoning_match.group(1).strip()
    
    # Calculate grade
    overall = parsed['scores']['overall']
    if overall >= 90:
        parsed['grade'] = 'A (Excellent)'
        parsed['pass_threshold'] = True
    elif overall >= 80:
        parsed['grade'] = 'B (Very Good)'
        parsed['pass_threshold'] = True
    elif overall >= 70:
        parsed['grade'] = 'C (Good)'
        parsed['pass_threshold'] = True
    elif overall >= 60:
        parsed['grade'] = 'D (Acceptable)'
        parsed['pass_threshold'] = False
    else:
        parsed['grade'] = 'F (Poor Fit)'
        parsed['pass_threshold'] = False
    
    return parsed

# Parse all candidate results
print("=" * 60)
print("PARSING ALL CANDIDATE ANALYSES")
print("=" * 60)

for result in matching_results['results']:
    parsed_data = parse_candidate_analysis(result['analysis'])
    
    # Add parsed data to result
    result['scores'] = parsed_data['scores']
    result['recommendation'] = parsed_data['recommendation']
    result['key_strength'] = parsed_data['key_strength']
    result['main_concern'] = parsed_data['main_concern']
    result['reasoning'] = parsed_data['reasoning']
    result['grade'] = parsed_data['grade']
    result['pass_threshold'] = parsed_data['pass_threshold']

# Sort by overall score
sorted_candidates = sorted(
    matching_results['results'],
    key=lambda x: x['scores']['overall'],
    reverse=True
)

print(f"Parsed {len(sorted_candidates)} candidate analyses")
print(f"Extracted scores, recommendations, and reasoning")
print("=" * 60)

PARSING ALL CANDIDATE ANALYSES
Parsed 50 candidate analyses
Extracted scores, recommendations, and reasoning


In [19]:
from datetime import datetime

print("=" * 80)
print("📊 VOLUNTEER COORDINATOR EXECUTIVE SUMMARY")
print("=" * 80)
print(f"/nPosition: {matching_results['job_info']['position']}")
print(f"Department: {matching_results['job_info']['department']}")
print(f"Volunteers Needed: {matching_results['job_info']['volunteers_needed']}")
print(f"Date: {datetime.now().strftime('%B %d, %Y')}")
print(f"Total Candidates Reviewed: {len(sorted_candidates)}")
print("/n" + "=" * 80)


strong_fits = sum(1 for c in sorted_candidates if c['scores']['overall'] >= 80)
potential_fits = sum(1 for c in sorted_candidates if 60 <= c['scores']['overall'] < 80)
weak_fits = sum(1 for c in sorted_candidates if c['scores']['overall'] < 60)

print(f"/nCANDIDATE POOL SUMMARY:")
print(f"Strong Fits (80-100):     {strong_fits} candidates")
print(f"Potential Fits (60-79):   {potential_fits} candidates")
print(f"Weak Fits (below 60):     {weak_fits} candidates")

print(f"/nTOP 10 RECOMMENDED CANDIDATES")
print("=" * 80)

for i, candidate in enumerate(sorted_candidates[:10], 1):
    print(f"/n{'─'*80}")
    print(f"#{i} | {candidate['filename']}")
    print(f"{'─'*80}")
    
    # Score badge
    overall = candidate['scores']['overall']
    if overall >= 90:
        emoji = "🌟"
    elif overall >= 75:
        emoji = "✅"
    elif overall >= 60:
        emoji = "👍"
    else:
        emoji = "⚠️"
    
    print(f"/n{emoji} OVERALL SCORE: {overall}/100 - {candidate['grade']}")
    print(f"   Recommendation: {candidate['recommendation']}")
    
    # Score breakdown
    print(f"/n📊 Score Breakdown:")
    print(f"   • Education Match:     {candidate['scores']['education']}/100")
    print(f"   • Experience Match:    {candidate['scores']['experience']}/100")
    print(f"   • Skills Match:        {candidate['scores']['skills']}/100")
    print(f"   • Communication Fit:   {candidate['scores']['communication']}/100")
    
    # Contact
    print(f"/n   Contact Information:")
    print(f"   • Email: {candidate['metadata']['email'] or 'Not provided'}")
    print(f"   • Phone: {candidate['metadata']['phone'] or 'Not provided'}")
    
    # Why this candidate
    print(f"/n✅ WHY THIS CANDIDATE STANDS OUT:")
    print(f"   {candidate['key_strength']}")
    
    print(f"/n⚠️  POTENTIAL CONCERN:")
    print(f"   {candidate['main_concern']}")
    
    print(f"/n💡 AI ANALYST'S REASONING:")
    print(f"   {candidate['reasoning']}")

print("/n" + "=" * 80)
print("END OF EXECUTIVE SUMMARY")
print("=" * 80)

📊 VOLUNTEER COORDINATOR EXECUTIVE SUMMARY
/nPosition: Community Outreach Support Volunteer
DEPARTMENT: Mathematics
Volunteers Needed: 2
ORGANIZATIONAL LEADERS:
- Dr. Andrew Parker
- Prof. John Carter
- Dr. Olivia Scott
ABOUT THE ROLE:
In this role, you will support departmental initiatives and foster community engagement. Volunteers will contribute to improving student experience and supporting academic success.
REQUIREMENTS:
✓ Enrolled in or completed a relevant academic program
✓ Strong communication and collaboration skills
✓ Ability to work with diverse student populations
✓ Reliable and organized with strong time management
PREFERRED QUALIFICATIONS:
✓ Leadership experience or past volunteer work
✓ Familiarity with campus resources and student groups
✓ Experience with event support or mentoring
RESPONSIBILITIES:
 Assist students, staff, or faculty depending on assignment
 Support program operations and outreach activities
 Track engagement and provide feedback to supervisors
 P

In [20]:
print("=" * 120)
print("📋 TOP 10 CANDIDATES - COMPARISON TABLE")
print("=" * 120)
print(f"Position: {matching_results['job_info']['position']}/n")

# Header
print(f"{'Rank':<6}{'Candidate':<30}{'Score':<10}{'Grade':<8}{'Recommendation':<18}{'Key Strength':<50}")
print("─" * 120)

# Data
for i, candidate in enumerate(sorted_candidates[:10], 1):
    name = candidate['filename'][:28]
    score = f"{candidate['scores']['overall']}/100"
    grade = candidate['grade'].split()[0]
    rec = candidate['recommendation'][:15]
    strength = candidate['key_strength'][:47] + "..." if len(candidate['key_strength']) > 47 else candidate['key_strength']
    
    print(f"{i:<6}{name:<30}{score:<10}{grade:<8}{rec:<18}{strength:<50}")

print("=" * 120)

📋 TOP 10 CANDIDATES - COMPARISON TABLE
Position: Community Outreach Support Volunteer
DEPARTMENT: Mathematics
Volunteers Needed: 2
ORGANIZATIONAL LEADERS:
- Dr. Andrew Parker
- Prof. John Carter
- Dr. Olivia Scott
ABOUT THE ROLE:
In this role, you will support departmental initiatives and foster community engagement. Volunteers will contribute to improving student experience and supporting academic success.
REQUIREMENTS:
✓ Enrolled in or completed a relevant academic program
✓ Strong communication and collaboration skills
✓ Ability to work with diverse student populations
✓ Reliable and organized with strong time management
PREFERRED QUALIFICATIONS:
✓ Leadership experience or past volunteer work
✓ Familiarity with campus resources and student groups
✓ Experience with event support or mentoring
RESPONSIBILITIES:
 Assist students, staff, or faculty depending on assignment
 Support program operations and outreach activities
 Track engagement and provide feedback to supervisors
 Partic

In [ ]:
"Run specific uploaded resume to receive overall with all opportunities" 
"Triggers background job for that one resume against all opportunities"
"New opportunity assesses all resumes scores resumes"
"If resume hits threshold then remove from opportunity"

In [21]:
"""
Test script for complete scoring system - ALL resumes x ALL opportunities
"""

import logging
from pathlib import Path

from storage_manager import StorageManager
from incremental_scorer import IncrementalScorer, ScoringError
from results_generator import ResultsGenerator

# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

logger = logging.getLogger(__name__)


def test_complete_scoring():
    """Test complete scoring: ALL resumes against ALL opportunities"""
    
    print("\n" + "=" * 120)
    print("🤖 COMPLETE SCORING SYSTEM TEST")
    print("=" * 120)
    
    # Initialize components
    try:
        storage = StorageManager('.')
        scorer = IncrementalScorer(storage, threshold=70)
        results_gen = ResultsGenerator(storage)
        
        print("✓ All components initialized\n")
    except Exception as e:
        print(f"✗ Initialization failed: {e}")
        return
    
    # Check what we have
    all_resumes = storage.get_all_resumes()
    all_opps = storage.get_all_opportunities()
    
    if not all_resumes:
        print("⚠️  No resumes found. Run test_storage_manager.py first!")
        return
    
    if not all_opps:
        print("⚠️  No opportunities found. Run test_storage_manager.py first!")
        return
    
    print(f"📊 Current Database:")
    print(f"   Resumes: {len(all_resumes)}")
    print(f"   Opportunities: {len(all_opps)}")
    print(f"   Total pairs to score: {len(all_resumes) * len(all_opps)}\n")
    
    # Step 1: Score everything
    print("=" * 120)
    print("STEP 1: Score All Resumes Against All Opportunities")
    print("=" * 120)
    
    try:
        print("⏳ Starting complete scoring... (this will take several minutes)\n")
        
        results = scorer.score_all_resumes_all_opportunities(force=False)
        
        print("\n✓ Complete scoring finished!")
        
    except ScoringError as e:
        print(f"✗ Scoring failed: {e}")
        return
    except Exception as e:
        print(f"✗ Unexpected error: {e}")
        return
    
    # Step 2: Display top 10 for each opportunity
    print("\n" + "=" * 120)
    print("STEP 2: Display Top 10 Candidates for Each Opportunity")
    print("=" * 120 + "\n")
    
    results_gen.display_all_opportunities_top_10()
    
    # Step 3: Check for overlap
    print("\n" + "=" * 120)
    print("STEP 3: Analyze Candidate Overlap")
    print("=" * 120 + "\n")
    
    results_gen.display_overlap_report()
    
    # Step 4: Export reports for opportunity managers
    print("\n" + "=" * 120)
    print("STEP 4: Export Individual Opportunity Reports")
    print("=" * 120 + "\n")
    
    try:
        results_gen.export_all_opportunity_reports('results/opportunity_reports')
    except Exception as e:
        print(f"✗ Export failed: {e}")
    
    # Step 5: Check for low-scoring resumes
    print("\n" + "=" * 120)
    print("STEP 5: Identify Low-Scoring Resumes (below 70 on ALL opportunities)")
    print("=" * 120 + "\n")
    
    low_scorers = storage.get_low_scoring_resumes(threshold=70)
    
    if low_scorers:
        print(f"Found {len(low_scorers)} resumes scoring below 70 on ALL opportunities:\n")
        for resume in low_scorers:
            print(f"  📄 {resume['filename']} (ID: {resume['resume_id']})")
            if resume['email']:
                print(f"     Email: {resume['email']}")
            if resume['phone']:
                print(f"     Phone: {resume['phone']}")
            print(f"     Max Score: {max(s['overall'] for s in resume['scores'].values())}/100\n")
    else:
        print("✓ No resumes scored below 70 on all opportunities")
    
    # Final statistics
    print("\n" + "=" * 120)
    print("📊 FINAL STATISTICS")
    print("=" * 120)
    
    storage.update_metadata()
    metadata = storage.get_metadata()
    
    print(f"\nSystem Status:")
    print(f"  📄 Total Resumes: {metadata['total_resumes']}")
    print(f"  💼 Total Opportunities: {metadata['total_opportunities']}")
    print(f"  📊 Total Scores: {metadata['total_scores']}")
    
    if metadata['total_resumes'] > 0 and metadata['total_opportunities'] > 0:
        max_possible = metadata['total_resumes'] * metadata['total_opportunities']
        coverage = (metadata['total_scores'] / max_possible) * 100
        print(f"  📈 Coverage: {coverage:.1f}% ({metadata['total_scores']}/{max_possible})")
    
    print("\n" + "=" * 120)
    print("✓ TEST COMPLETED SUCCESSFULLY!")
    print("=" * 120)
    print("\n📁 Output Files:")
    print("   • results/opportunity_reports/ - Individual JSON reports for each opportunity")
    print("   • results/*.json - Database files\n")


if __name__ == "__main__":
    test_complete_scoring()

2025-11-07 13:43:23,480 - INFO - Configuration validated successfully



🤖 COMPLETE SCORING SYSTEM TEST


2025-11-07 13:43:23,698 - INFO - ================================================================================
2025-11-07 13:43:23,699 - INFO - Starting complete scoring: 50 resumes x 100 opportunities
2025-11-07 13:43:23,700 - INFO - Total scoring pairs: 5000
2025-11-07 13:43:23,700 - INFO - ================================================================================
2025-11-07 13:43:23,700 - INFO - 
📋 Scoring for: Community Outreach Support Volunteer
DEPARTMENT: Mathematics
Volunteers Needed: 2
ORGANIZATIONAL LEADERS:
- Dr. Andrew Parker
- Prof. John Carter
- Dr. Olivia Scott
ABOUT THE ROLE:
In this role, you will support departmental initiatives and foster community engagement. Volunteers will contribute to improving student experience and supporting academic success.
REQUIREMENTS:
✓ Enrolled in or completed a relevant academic program
✓ Strong communication and collaboration skills
✓ Ability to work with diverse student populations
✓ Reliable and organized with strong time m

✓ All components initialized

📊 Current Database:
   Resumes: 50
   Opportunities: 100
   Total pairs to score: 5000

STEP 1: Score All Resumes Against All Opportunities
⏳ Starting complete scoring... (this will take several minutes)



2025-11-07 13:43:25,743 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-11-07 13:43:25,748 - INFO - Scored Resume 1 x Opportunity 2 = 82/100
2025-11-07 13:43:27,456 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-11-07 13:43:27,471 - INFO - Scored Resume 2 x Opportunity 2 = 78/100
2025-11-07 13:43:29,839 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-11-07 13:43:29,846 - INFO - Scored Resume 3 x Opportunity 2 = 75/100
2025-11-07 13:43:31,846 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-11-07 13:43:31,849 - INFO - Scored Resume 4 x Opportunity 2 = 75/100
2025-11-07 13:43:33,985 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-11-07 13:43:33,995 - INFO - Scored Resume 5 x Opportunity 2 = 80/100
2025-11-07 13:43:33,995 - INFO -    Progress: 5/50 resumes | Over


✓ Complete scoring finished!

STEP 2: Display Top 10 Candidates for Each Opportunity


🎯 ALL OPPORTUNITIES - TOP 10 CANDIDATES PER OPPORTUNITY
Total Opportunities: 100

📋 TOP 10 CANDIDATES - COMPARISON TABLE
Position: Community Outreach Support Volunteer
DEPARTMENT: Mathematics
Volunteers Needed: 2
ORGANIZATIONAL LEADERS:
- Dr. Andrew Parker
- Prof. John Carter
- Dr. Olivia Scott
ABOUT THE ROLE:
In this role, you will support departmental initiatives and foster community engagement. Volunteers will contribute to improving student experience and supporting academic success.
REQUIREMENTS:
✓ Enrolled in or completed a relevant academic program
✓ Strong communication and collaboration skills
✓ Ability to work with diverse student populations
✓ Reliable and organized with strong time management
PREFERRED QUALIFICATIONS:
✓ Leadership experience or past volunteer work
✓ Familiarity with campus resources and student groups
✓ Experience with event support or mentoring
RESPONSIBILITIES:
 Assis

In [1]:
"""
Quick check of all registered resumes
"""

from storage_manager import StorageManager

storage = StorageManager('.')
resumes = storage.get_all_resumes()

print("=" * 80)
print(f"📄 REGISTERED RESUMES ({len(resumes)} total)")
print("=" * 80)

for resume_id, resume_data in sorted(resumes.items()):
    print(f"\nID {resume_id}: {resume_data['filename']}")
    print(f"   Registered: {resume_data.get('registered_at', 'Unknown')}")
    
    # Check scores
    scores = storage.get_resume_scores(resume_id)
    print(f"   Scores: {len(scores)}")
    
    if len(scores) == 0:
        print(f"   ⚠️ NOT SCORED YET!")

print("\n" + "=" * 80)

📄 REGISTERED RESUMES (51 total)

ID 1: Aimee_Montoya_Resume.txt
   Registered: 2025-11-07T13:12:11.722516
   Scores: 100

ID 2: Allison_Hill_Resume.txt
   Registered: 2025-11-07T13:12:11.723678
   Scores: 100

ID 3: Andrew_Graham_Resume.docx
   Registered: 2025-11-07T13:12:11.735516
   Scores: 100

ID 4: Angela_Dennis_Resume.txt
   Registered: 2025-11-07T13:12:11.737228
   Scores: 100

ID 5: Angela_Hernandez_Resume.pdf
   Registered: 2025-11-07T13:12:11.740536
   Scores: 100

ID 6: Ashley_James_Resume.pdf
   Registered: 2025-11-07T13:12:11.743213
   Scores: 100

ID 7: Brittany_Anderson_Resume.pdf
   Registered: 2025-11-07T13:12:11.745399
   Scores: 100

ID 8: Cameron_Dean_Resume.docx
   Registered: 2025-11-07T13:12:11.753956
   Scores: 100

ID 9: Cameron_Fisher_Resume.txt
   Registered: 2025-11-07T13:12:11.755759
   Scores: 100

ID 10: Cindy_Anderson_Resume.pdf
   Registered: 2025-11-07T13:12:11.757930
   Scores: 100

ID 11: Cody_Larson_Resume.pdf
   Registered: 2025-11-07T13:12:11.759

In [2]:
"""
Check Ethan Clark's scores and ranking
"""

from storage_manager import StorageManager
import json

storage = StorageManager('.')

print("=" * 100)
print("🔍 CHECKING ETHAN CLARK")
print("=" * 100)

# Find Ethan Clark in resumes
all_resumes = storage.get_all_resumes()
ethan_id = None

for resume_id, resume_data in all_resumes.items():
    if 'ethan' in resume_data['filename'].lower() or 'clark' in resume_data['filename'].lower():
        ethan_id = resume_id
        print(f"\n✓ Found Ethan Clark:")
        print(f"   Resume ID: {resume_id}")
        print(f"   Filename: {resume_data['filename']}")
        break

if not ethan_id:
    print("\n✗ Ethan Clark not found in database!")
    print("\nRegistered resumes:")
    for resume_id, resume_data in all_resumes.items():
        print(f"   ID {resume_id}: {resume_data['filename']}")
    exit()

# Check his scores
scores = storage.get_resume_scores(ethan_id)
print(f"\n📊 Total scores: {len(scores)}")

# Check opportunity 2 specifically
if 2 in scores:
    opp2_score = scores[2]
    print(f"\n✓ Opportunity 2 score found:")
    print(f"   Overall: {opp2_score['overall']}/100")
    print(f"   Skills: {opp2_score.get('skills_match', 'N/A')}")
    print(f"   Experience: {opp2_score.get('experience_match', 'N/A')}")
    print(f"   Grade: {opp2_score.get('grade', 'N/A')}")
else:
    print(f"\n✗ No score for Opportunity 2!")

# Get all scores for opportunity 2
print("\n" + "=" * 100)
print("📋 OPPORTUNITY 2 - ALL CANDIDATES")
print("=" * 100)

opp2_scores = storage.get_opportunity_scores(2)
print(f"\nTotal candidates scored: {len(opp2_scores)}")

# Build candidate list with scores
candidates = []
for resume_id, score_data in opp2_scores.items():
    resume = storage.get_resume(resume_id)
    if resume:
        candidates.append({
            'resume_id': resume_id,
            'filename': resume['filename'],
            'overall': score_data.get('overall', 0)
        })

# Sort by score
sorted_candidates = sorted(candidates, key=lambda x: x['overall'], reverse=True)

print("\n🏆 TOP 15 CANDIDATES (sorted by score):")
for i, candidate in enumerate(sorted_candidates[:15], 1):
    highlight = "👉 " if candidate['resume_id'] == ethan_id else "   "
    print(f"{highlight}{i:2}. Resume {candidate['resume_id']:3}: {candidate['overall']:3}/100 - {candidate['filename'][:50]}")

# Check if Ethan is in top 10
ethan_rank = None
for i, candidate in enumerate(sorted_candidates, 1):
    if candidate['resume_id'] == ethan_id:
        ethan_rank = i
        break

if ethan_rank:
    if ethan_rank <= 10:
        print(f"\n✓ Ethan Clark ranks #{ethan_rank} for Opportunity 2")
    else:
        print(f"\n⚠️  Ethan Clark ranks #{ethan_rank} for Opportunity 2 (outside top 10)")
else:
    print(f"\n✗ Ethan Clark not found in rankings!")

# Check the actual JSON report file
print("\n" + "=" * 100)
print("📄 CHECKING OPPORTUNITY 2 JSON REPORT FILE")
print("=" * 100)

from pathlib import Path
import glob

report_files = glob.glob('results/opportunity_reports/opportunity_2_*.json')
if report_files:
    report_file = report_files[0]
    print(f"\nReport file: {report_file}")
    
    with open(report_file, 'r') as f:
        report_data = json.load(f)
    
    print(f"Total candidates in report: {len(report_data['all_candidates'])}")
    print(f"Top 10 candidates in report: {len(report_data['top_10_candidates'])}")
    
    print("\n🏆 TOP 10 IN JSON FILE:")
    for i, candidate in enumerate(report_data['top_10_candidates'], 1):
        highlight = "👉 " if 'ethan' in candidate['filename'].lower() or 'clark' in candidate['filename'].lower() else "   "
        print(f"{highlight}{i:2}. Resume {candidate['resume_id']:3}: {candidate['overall_score']:3}/100 - {candidate['filename'][:50]}")
    
    # Check if Ethan is in the full list
    ethan_in_report = False
    for candidate in report_data['all_candidates']:
        if 'ethan' in candidate['filename'].lower() or 'clark' in candidate['filename'].lower():
            ethan_in_report = True
            print(f"\n✓ Ethan Clark IS in the full report")
            break
    
    if not ethan_in_report:
        print(f"\n✗ Ethan Clark NOT in the report - report needs regeneration!")
else:
    print("\n✗ No report file found for Opportunity 2")

print("\n" + "=" * 100)

🔍 CHECKING ETHAN CLARK

✓ Found Ethan Clark:
   Resume ID: 51
   Filename: ethan_clark_resume.pdf

📊 Total scores: 100

✓ Opportunity 2 score found:
   Overall: 85/100
   Skills: 80
   Experience: 75
   Grade: B

📋 OPPORTUNITY 2 - ALL CANDIDATES

Total candidates scored: 51

🏆 TOP 15 CANDIDATES (sorted by score):
👉  1. Resume  51:  85/100 - ethan_clark_resume.pdf
    2. Resume   1:  82/100 - Aimee_Montoya_Resume.txt
    3. Resume   5:  80/100 - Angela_Hernandez_Resume.pdf
    4. Resume   2:  78/100 - Allison_Hill_Resume.txt
    5. Resume  15:  78/100 - Dr._Steven_Martin_Resume.pdf
    6. Resume   3:  75/100 - Andrew_Graham_Resume.docx
    7. Resume   4:  75/100 - Angela_Dennis_Resume.txt
    8. Resume   6:  75/100 - Ashley_James_Resume.pdf
    9. Resume   7:  75/100 - Brittany_Anderson_Resume.pdf
   10. Resume  10:  75/100 - Cindy_Anderson_Resume.pdf
   11. Resume  16:  75/100 - Edward_Novak_Resume.txt
   12. Resume  17:  75/100 - Emily_Green_Resume.pdf
   13. Resume  20:  75/100 - Jen

In [2]:
from storage_manager import StorageManager
import json

storage = StorageManager('.')

# Load scores
scores_db = storage.load_json(storage.scores_db)

# Remove resume 51's scores
if '51' in scores_db['scores']:
    del scores_db['scores']['51']
    storage.save_json(storage.scores_db, scores_db)
    print("✓ Removed Ethan's scores")

# Load resumes
resumes_db = storage.load_json(storage.resumes_db)

# Remove resume 51
if '51' in resumes_db['resumes']:
    del resumes_db['resumes']['51']
    storage.save_json(storage.resumes_db, resumes_db)
    print("✓ Removed Ethan from database")